In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
import shapely

In [ ]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

In [ ]:
df_geographic_unique = df[['state', 'reportedZipCode', 'countyCode', 'censusTract', 'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss']].drop_duplicates()

In [ ]:
df_geographic_unique = df_geographic_unique.dropna(subset=['latitude', 'censusBlockGroupFips'])

In [ ]:
df_geographic_unique['censusBlockGroupFips'] = [str(int(float(i))) for i in df_geographic_unique['censusBlockGroupFips']]
df_geographic_unique['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_geographic_unique['censusBlockGroupFips']]

In [ ]:
# Define bins and labels for yearOfLoss_1980_2020
bins_1980_2020 = [df_geographic_unique['yearOfLoss'].min(), 1990, 2000, 2010, 2020, df_geographic_unique['yearOfLoss'].max() + 1]
labels_1980_2020 = [1980, 1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_1980_2020'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1980_2020, labels=labels_1980_2020, right=False).astype(int)

In [ ]:
# states = pygris.states()
# states_list = sorted(states['STUSPS'].unique())

In [ ]:
# BG_list_2000 = []

# for i in states_list:
#     BG_list_2000.append(pygris.block_groups(state = i, year = 2000))

In [ ]:
BG_list_df2000 = pygris.block_groups(state = 'NC', year = 2000)

In [ ]:
# BG_list_1990 = []

# for i in states_list:
#     BG_list_1990.append(pygris.block_groups(state = i, year = 1990, cb = True))

In [ ]:
BG_list_df1990 = pygris.block_groups(state = 'NC', year = 1990, cb = True)

In [ ]:
# BG_list_df2000 = pd.concat(BG_list_2000, ignore_index=True)
# BG_list_df1990 = pd.concat(BG_list_1990, ignore_index=True)

In [ ]:
BG_list_df2000 = BG_list_df2000[['BKGPIDFP00', 'geometry']]
BG_list_df2000['year'] = 2000

BG_list_df2000 = BG_list_df2000.rename(columns={"BKGPIDFP00": "GEOID"})

BG_list_df1990 = BG_list_df1990[['GEOID', 'geometry']]
BG_list_df1990['year'] = 1990

#### Checking for 2000...

In [ ]:
# df_geographic_unique2000 = df_geographic_unique[df_geographic_unique['yearOfLoss_1980_2020']==2000]

In [ ]:
df_geographic_unique2000 = df_geographic_unique[(df_geographic_unique['yearOfLoss_1980_2020']==2000) & (df_geographic_unique['state']=='NC')]

In [ ]:
# Read the parquet file
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/lat_long_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
lat_long_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

In [ ]:
lat_long_df.rename(columns={'geometry': 'geometry_lat_long'}, inplace=True)

In [ ]:
# Setting the multi-index on lat_long_df
lat_long_df.set_index(['latitude', 'longitude'], inplace=True)

# Mapping the values
df_geographic_unique2000['geometry_lat_long'] = df_geographic_unique2000.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])

# Resetting the index of lat_long_df (optional, but good practice)
lat_long_df.reset_index(inplace=True)

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon
from itertools import product
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Create an empty GeoDataFrame to store the intersection results
new_unit_df2000 = gpd.GeoDataFrame(columns=['reportedZipCode', 'countyCode', 'censusTract',
                                       'censusBlockGroupFips', 'MATCHED_censusBlockGroupFips', 'latitude', 'longitude', 'year', 'state', 'geometry_BG',
                                        'geometry_lat_long', 'new_geometry'])

# Iterate through each row in BG_df_2000 and each row in lat_long_df to find intersections
for idx_unit_data, row_unit_data in df_geographic_unique2000.iterrows():
    for idx_unit_shape, row_unit_shape in BG_list_df2000.iterrows():
        year = row_unit_data['yearOfLoss']
        year2 = row_unit_data['yearOfLoss_1980_2020']
        bg_id = row_unit_data['censusBlockGroupFips']
        bg_id2 = row_unit_shape['GEOID']
        bg_geometry = row_unit_shape['geometry']
        tract_id = row_unit_data['censusTract']
        county_id = row_unit_data['countyCode']
        state = row_unit_data['state']
        lat_long_geometry = row_unit_data['geometry_lat_long']
        lat = row_unit_data['latitude']
        long = row_unit_data['longitude']
        zipcode = row_unit_data['reportedZipCode']
    
        # Compute intersection geometry
        intersection_geometry = bg_geometry.intersection(lat_long_geometry)
    
        new_unit_df2000 = pd.concat([new_unit_df2000, pd.DataFrame({
            'reportedZipCode': [zipcode],
            'countyCode': [county_id],
            'censusTract': [tract_id],
            'censusBlockGroupFips': [bg_id],
            'MATCHED_censusBlockGroupFips': [bg_id2],
            'latitude': [lat],
            'longitude': [long],
            'year': [year],
            'decade': [year2],
            'state': [state],
            'geometry_BG': [bg_geometry],
            'geometry_lat_long': [lat_long_geometry],
            'new_geometry': [intersection_geometry]
        })], ignore_index=True)

In [ ]:
new_unit_df2000

#### Checking for 1990

In [ ]:
# df_geographic_unique1990 = df_geographic_unique[df_geographic_unique['yearOfLoss_1980_2020']==1990]

In [ ]:
df_geographic_unique1990= df_geographic_unique[(df_geographic_unique['yearOfLoss_1980_2020']==1990) & (df_geographic_unique['state']=='NC')]

In [ ]:
# Setting the multi-index on lat_long_df
lat_long_df.set_index(['latitude', 'longitude'], inplace=True)

# Mapping the values
df_geographic_unique1990['geometry_lat_long'] = df_geographic_unique1990.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])

# Resetting the index of lat_long_df (optional, but good practice)
lat_long_df.reset_index(inplace=True)

In [ ]:
# Create an empty GeoDataFrame to store the intersection results
new_unit_df1990 = gpd.GeoDataFrame(columns=['reportedZipCode', 'countyCode', 'censusTract',
                                       'censusBlockGroupFips', 'MATCHED_censusBlockGroupFips', 'latitude', 'longitude', 'year', 'state', 'geometry_BG',
                                        'geometry_lat_long', 'new_geometry'])

# Iterate through each row in BG_df_2000 and each row in lat_long_df to find intersections
for idx_unit_data, row_unit_data in df_geographic_unique2000.iterrows():
    for idx_unit_shape, row_unit_shape in BG_list_df2000.iterrows():
        year = row_unit_data['yearOfLoss']
        year2 = row_unit_data['yearOfLoss_1980_2020']
        bg_id = row_unit_data['censusBlockGroupFips']
        bg_id2 = row_unit_shape['GEOID']
        bg_geometry = row_unit_shape['geometry']
        tract_id = row_unit_data['censusTract']
        county_id = row_unit_data['countyCode']
        state = row_unit_data['state']
        lat_long_geometry = row_unit_data['geometry_lat_long']
        lat = row_unit_data['latitude']
        long = row_unit_data['longitude']
        zipcode = row_unit_data['reportedZipCode']
    
        # Compute intersection geometry
        intersection_geometry = bg_geometry.intersection(lat_long_geometry)
    
        new_unit_df1990 = pd.concat([new_unit_df1990, pd.DataFrame({
            'reportedZipCode': [zipcode],
            'countyCode': [county_id],
            'censusTract': [tract_id],
            'censusBlockGroupFips': [bg_id],
            'MATCHED_censusBlockGroupFips': [bg_id2],
            'latitude': [lat],
            'longitude': [long],
            'year': [year],
            'decade': [year2],
            'state': [state],
            'geometry_BG': [bg_geometry],
            'geometry_lat_long': [lat_long_geometry],
            'new_geometry': [intersection_geometry]
        })], ignore_index=True)